In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [2]:
train_data = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test_data = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

In [3]:
target_col = 'efs'  # Treating the "efs" column as a binary outcome for classification
X_train = train_data.drop(columns=[target_col, 'efs_time'])
y_train = train_data[target_col]

In [4]:
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

In [5]:
numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [6]:
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [7]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_preprocessor, numerical_cols),
    ('cat', categorical_preprocessor, categorical_cols)
])

In [8]:
X_train_preprocessed = preprocessor.fit_transform(X_train)

In [9]:
model = xgb.XGBRegressor(eval_metric="logloss")
model.fit(X_train_preprocessed, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='logloss',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [10]:
X_test = test_data
X_test_preprocessed = preprocessor.transform(X_test)

In [11]:
y_test_predictions = model.predict(X_test_preprocessed)

In [12]:
submission = pd.DataFrame({
    'ID': test_data['ID'],
    'prediction': y_test_predictions
})

In [13]:
submission

,ID,prediction
0,28800,0.179912
1,28801,0.870581
2,28802,0.244291


In [14]:
submission.to_csv('submission.csv', index=False)